# **Deep Fake Detection on celeb df v2 dataset using BNNs**

In [1]:
!pip install --upgrade kagglehub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.4
    Uninstalling kagglehub-0.3.4:
      Successfully uninstalled kagglehub-0.3.4


Calling the dataset using kagglehub

In [2]:
import kagglehub
import os

# Download the dataset
dataset_path = kagglehub.dataset_download("reubensuju/celeb-df-v2")

# Print the path to confirm where the dataset is stored
print("Path to dataset files:", dataset_path)


100%|██████████| 9.29G/9.29G [01:52<00:00, 88.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/reubensuju/celeb-df-v2/versions/1


In [3]:
pip install torch torchvision opencv-python matplotlib numpy brevitas kaggle


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.6/706.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 50.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


## To check where is the kaggle dataset getitng stored then moving it to content folder for better and easier handling for the dataset

In [4]:
import os

# Use the actual KaggleHub path
dataset_path = "/root/.cache/kagglehub/datasets/reubensuju/celeb-df-v2/versions/1"

# Print and check if the path exists
if os.path.exists(dataset_path):
    print("Dataset found at:", dataset_path)
    print("Contents of the dataset folder:")
    for root, dirs, files in os.walk(dataset_path):
        print(f"Directory: {root}")
        print(f"Files: {files[:5]}")  # Print a sample of 5 files
else:
    print("Dataset path does not exist. Check your KaggleHub download step.")


Dataset found at: /root/.cache/kagglehub/datasets/reubensuju/celeb-df-v2/versions/1
Contents of the dataset folder:
Directory: /root/.cache/kagglehub/datasets/reubensuju/celeb-df-v2/versions/1
Files: ['List_of_testing_videos.txt']
Directory: /root/.cache/kagglehub/datasets/reubensuju/celeb-df-v2/versions/1/Celeb-real
Files: ['id43_0000.mp4', 'id21_0007.mp4', 'id13_0010.mp4', 'id2_0008.mp4', 'id31_0008.mp4']
Directory: /root/.cache/kagglehub/datasets/reubensuju/celeb-df-v2/versions/1/YouTube-real
Files: ['00257.mp4', '00189.mp4', '00002.mp4', '00135.mp4', '00150.mp4']
Directory: /root/.cache/kagglehub/datasets/reubensuju/celeb-df-v2/versions/1/Celeb-synthesis
Files: ['id52_id50_0002.mp4', 'id26_id22_0004.mp4', 'id25_id28_0010.mp4', 'id29_id34_0005.mp4', 'id17_id35_0007.mp4']


## copying dataset to content folder

In [5]:
!cp -r /root/.cache/kagglehub/datasets/reubensuju/celeb-df-v2/versions/1 /content/celeb_df_v2


In [6]:
# Parse the List_of_testing_videos.txt file
test_videos = []

with open('/content/celeb_df_v2/List_of_testing_videos.txt', 'r') as f:
    for line in f:
        test_videos.append(line.strip())  # Remove spaces, newlines, etc.

print("Number of testing videos:", len(test_videos))
print("Sample test videos:", test_videos[:5])


Number of testing videos: 518
Sample test videos: ['1 YouTube-real/00170.mp4', '1 YouTube-real/00208.mp4', '1 YouTube-real/00063.mp4', '1 YouTube-real/00024.mp4', '1 YouTube-real/00021.mp4']


In [7]:
import os
import shutil
import cv2
from sklearn.model_selection import train_test_split


In [27]:
# Print a few test video names
print("Test Video Names from List:")
print(test_videos[:10])


Test Video Names from List:
['1 YouTube-real/00170.mp4', '1 YouTube-real/00208.mp4', '1 YouTube-real/00063.mp4', '1 YouTube-real/00024.mp4', '1 YouTube-real/00021.mp4', '1 YouTube-real/00036.mp4', '1 YouTube-real/00202.mp4', '1 YouTube-real/00236.mp4', '1 YouTube-real/00197.mp4', '1 YouTube-real/00133.mp4']


# Normalizing test data videos from the .txt file to extract those videos succesfully to the folders(Data preparation)

In [8]:
# Clean and normalize test video names
test_videos = []

with open('/content/celeb_df_v2/List_of_testing_videos.txt', 'r') as f:
    for line in f:
        # Split and extract the part after '/' and remove any spaces/newlines
        parts = line.strip().split('/')
        if len(parts) > 1:
            test_videos.append(parts[-1].strip().lower())  # Extract filename (last part)

print("Cleaned Test Video Names:")
print(test_videos[:10])  # Display the cleaned video names


Cleaned Test Video Names:
['00170.mp4', '00208.mp4', '00063.mp4', '00024.mp4', '00021.mp4', '00036.mp4', '00202.mp4', '00236.mp4', '00197.mp4', '00133.mp4']


# Spilitting the dataset into test and train

In [9]:
import shutil
import os

def organize_videos(input_folder, output_train, output_test, test_videos, label):
    """
    Organize videos into train and test folders based on the cleaned test video list.
    """
    os.makedirs(os.path.join(output_train, label), exist_ok=True)
    os.makedirs(os.path.join(output_test, label), exist_ok=True)

    normalized_test_videos = [name.strip().lower() for name in test_videos]  # Clean test video names

    for video in os.listdir(input_folder):
        video_name = os.path.basename(video).strip().lower()  # Normalize video name
        src = os.path.join(input_folder, video)

        # Compare with cleaned test list
        if video_name in normalized_test_videos:
            dst = os.path.join(output_test, label, video)
        else:
            dst = os.path.join(output_train, label, video)

        shutil.copy(src, dst)

# Define input folders (original dataset)
celeb_real_path = '/content/celeb_df_v2/Celeb-real'
celeb_synth_path = '/content/celeb_df_v2/Celeb-synthesis'
youtube_real_path = '/content/celeb_df_v2/YouTube-real'

# Define output folders
train_output = '/content/data/train'
test_output = '/content/data/test'

# Delete old data folder
if os.path.exists('/content/data'):
    shutil.rmtree('/content/data')

# Organize Celeb-real (Real videos)
organize_videos(celeb_real_path, train_output, test_output, test_videos, 'real')

# Organize Celeb-synthesis (Fake videos)
organize_videos(celeb_synth_path, train_output, test_output, test_videos, 'fake')

# Organize YouTube-real (Real videos)
organize_videos(youtube_real_path, train_output, test_output, test_videos, 'real')

print("Videos successfully split into train and test folders.")


Videos successfully split into train and test folders.


In [10]:
print("Train Real Videos:", len(os.listdir('/content/data/train/real')))
print("Train Fake Videos:", len(os.listdir('/content/data/train/fake')))
print("Test Real Videos:", len(os.listdir('/content/data/test/real')))
print("Test Fake Videos:", len(os.listdir('/content/data/test/fake')))


Train Real Videos: 712
Train Fake Videos: 5299
Test Real Videos: 178
Test Fake Videos: 340


# Due to limited hardware resources we are reducing the dataset's sizes(only the train/fake dataset from 5300 to 1500 video samples) before frame extraction to overcome processing time and smartly utilize our resources

In [13]:
import os
import random
import shutil

def reduce_videos(input_folder, output_folder, num_samples=1500):
    """
    Randomly select 'num_samples' videos from input_folder and copy them to output_folder.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    all_videos = os.listdir(input_folder)
    print(f"Total videos in '{input_folder}': {len(all_videos)}")

    # Randomly select 'num_samples' videos
    selected_videos = random.sample(all_videos, num_samples)

    # Copy selected videos to the output folder
    for video in selected_videos:
        src = os.path.join(input_folder, video)
        dst = os.path.join(output_folder, video)
        shutil.copy(src, dst)

    print(f"Successfully copied {num_samples} videos to '{output_folder}'.")

# Paths
train_fake_input = "/content/data/train/fake"
train_fake_output = "/content/data/reduced_train/fake"

# Reduce fake videos to 1500
reduce_videos(train_fake_input, train_fake_output, num_samples=1500)


Total videos in '/content/data/train/fake': 5299
Successfully copied 1500 videos to '/content/data/reduced_train/fake'.


In [14]:
print("Number of fake videos in reduced folder:",
      len(os.listdir('/content/data/reduced_train/fake')))


Number of fake videos in reduced folder: 1500


# Extracting frames using the frame rate of 1 frame for every 10 frames

In [15]:
import cv2
import os

def extract_frames_from_folder(input_folder, output_folder, frame_rate=10):
    """
    Extract frames from all videos in the input folder and save them as images.
    :param input_folder: Path to the folder containing videos.
    :param output_folder: Path to save extracted frames.
    :param frame_rate: Save one frame every 'frame_rate' frames.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for video_name in os.listdir(input_folder):
        video_path = os.path.join(input_folder, video_name)
        video_output_folder = os.path.join(output_folder, video_name.split('.')[0])

        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)

        cap = cv2.VideoCapture(video_path)
        frame_count, count = 0, 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if count % frame_rate == 0:
                frame_path = os.path.join(video_output_folder, f"frame_{frame_count:04d}.jpg")
                cv2.imwrite(frame_path, frame)
                frame_count += 1
            count += 1

        cap.release()
        print(f"Frames extracted from {video_name} to {video_output_folder}")

# Paths to train and test folders
train_real_input = '/content/data/train/real'
train_fake_input = '/content/data/reduced_train/fake'  # Reduced fake videos
test_real_input = '/content/data/test/real'
test_fake_input = '/content/data/test/fake'

# Output paths for frames
train_real_output = '/content/data/frames/train/real'
train_fake_output = '/content/data/frames/train/fake'
test_real_output = '/content/data/frames/test/real'
test_fake_output = '/content/data/frames/test/fake'

# Extract frames for all folders
extract_frames_from_folder(train_real_input, train_real_output, frame_rate=10)
extract_frames_from_folder(train_fake_input, train_fake_output, frame_rate=10)
extract_frames_from_folder(test_real_input, test_real_output, frame_rate=10)
extract_frames_from_folder(test_fake_input, test_fake_output, frame_rate=10)

print("Frame extraction completed successfully!")


Frames extracted from id43_0000.mp4 to /content/data/frames/train/real/id43_0000
Frames extracted from id21_0007.mp4 to /content/data/frames/train/real/id21_0007
Frames extracted from id13_0010.mp4 to /content/data/frames/train/real/id13_0010
Frames extracted from id31_0008.mp4 to /content/data/frames/train/real/id31_0008
Frames extracted from id31_0000.mp4 to /content/data/frames/train/real/id31_0000
Frames extracted from id0_0004.mp4 to /content/data/frames/train/real/id0_0004
Frames extracted from id45_0009.mp4 to /content/data/frames/train/real/id45_0009
Frames extracted from 00189.mp4 to /content/data/frames/train/real/00189
Frames extracted from 00002.mp4 to /content/data/frames/train/real/00002
Frames extracted from 00135.mp4 to /content/data/frames/train/real/00135
Frames extracted from id0_0005.mp4 to /content/data/frames/train/real/id0_0005
Frames extracted from 00150.mp4 to /content/data/frames/train/real/00150
Frames extracted from 00166.mp4 to /content/data/frames/train/re

In [18]:
import os

def count_samples_in_nested_folders(base_dir):
    """
    Count the number of image files in nested subfolders for each class.
    :param base_dir: Path to the 'frames' directory.
    """
    for split in ['train', 'test']:
        print(f"\nCounting samples in '{split}' folder:")
        for label in ['real', 'fake']:
            folder_path = os.path.join(base_dir, split, label)
            total_frames = 0

            # Walk through all subdirectories and count image files
            for subfolder in os.listdir(folder_path):
                subfolder_path = os.path.join(folder_path, subfolder)
                if os.path.isdir(subfolder_path):  # Only count in subfolders
                    num_frames = len([f for f in os.listdir(subfolder_path)
                                      if f.endswith(('.jpg', '.png'))])
                    print(f"  {label.capitalize()} -> {subfolder}: {num_frames} frames")
                    total_frames += num_frames

            print(f"  Total '{label}' frames in '{split}': {total_frames}")

# Path to frames folder
frames_folder = '/content/data/frames'

# Count the number of samples in each folder
count_samples_in_nested_folders(frames_folder)



Counting samples in 'train' folder:
  Real -> id39_0008: 32 frames
  Real -> id10_0005: 32 frames
  Real -> id33_0004: 42 frames
  Real -> 00146: 73 frames
  Real -> 00058: 51 frames
  Real -> id32_0005: 44 frames
  Real -> id16_0004: 41 frames
  Real -> id8_0002: 53 frames
  Real -> 00066: 46 frames
  Real -> id56_0005: 47 frames
  Real -> id24_0005: 34 frames
  Real -> id30_0006: 37 frames
  Real -> 00241: 45 frames
  Real -> 00131: 36 frames
  Real -> id43_0009: 34 frames
  Real -> id55_0009: 61 frames
  Real -> 00162: 45 frames
  Real -> 00199: 48 frames
  Real -> id57_0000: 64 frames
  Real -> id10_0006: 18 frames
  Real -> 00153: 49 frames
  Real -> id0_0000: 47 frames
  Real -> id3_0007: 37 frames
  Real -> id19_0009: 35 frames
  Real -> id45_0007: 34 frames
  Real -> 00120: 47 frames
  Real -> 00103: 48 frames
  Real -> 00245: 47 frames
  Real -> id61_0009: 46 frames
  Real -> 00223: 48 frames
  Real -> 00200: 47 frames
  Real -> id12_0003: 38 frames
  Real -> id30_0008: 50 fr

# Flattening our extracted frames folder as PyTorch’s ImageFolder to work, we need to move all frames into a single directory under each class (real or fake), rather than nested subfolders.

In [19]:
import os
import shutil

def flatten_frames(base_dir):
    """
    Flatten frames from all subfolders (videos) into a single folder for each class (real/fake).
    :param base_dir: Path to the base frames folder.
    """
    for split in ['train', 'test']:
        print(f"\nFlattening frames for '{split}' set:")

        for label in ['real', 'fake']:
            # Define source and target directories
            src_folder = os.path.join(base_dir, split, label)
            target_folder = os.path.join(base_dir, f"{split}_flattened", label)

            # Create target directory if it doesn't exist
            os.makedirs(target_folder, exist_ok=True)

            # Move all frames from subfolders into the target folder
            for subfolder in os.listdir(src_folder):
                subfolder_path = os.path.join(src_folder, subfolder)

                if os.path.isdir(subfolder_path):  # Ensure it's a directory
                    for frame in os.listdir(subfolder_path):
                        if frame.endswith(('.jpg', '.png')):  # Only consider image files
                            frame_src = os.path.join(subfolder_path, frame)
                            frame_dst = os.path.join(target_folder, f"{subfolder}_{frame}")  # Rename to avoid duplicates
                            shutil.move(frame_src, frame_dst)

            print(f"  '{label}' frames flattened into: {target_folder}")

# Path to the frames folder
frames_base_dir = '/content/data/frames'

# Flatten the frames
flatten_frames(frames_base_dir)



Flattening frames for 'train' set:
  'real' frames flattened into: /content/data/frames/train_flattened/real
  'fake' frames flattened into: /content/data/frames/train_flattened/fake

Flattening frames for 'test' set:
  'real' frames flattened into: /content/data/frames/test_flattened/real
  'fake' frames flattened into: /content/data/frames/test_flattened/fake


# After flattening, the **train_flattened** dataset contains **Real Frames: 29,163** and **Fake Frames: 57,372**, while the **test_flattened** dataset contains **Real Frames: 7,116** and **Fake Frames: 12,939**. Since these sample sizes are significantly large and computationally intensive to process, we will reduce them to an optimal size of **5,000 training samples** (balanced between real and fake frames) and **1,000 testing samples** (for each of real and fake frames).

In [21]:
def count_flattened_frames(base_dir):
    """
    Count the total number of frames in each flattened folder.
    """
    for split in ['train_flattened', 'test_flattened']:
        print(f"\nCounting frames in '{split}' folder:")

        for label in ['real', 'fake']:
            folder_path = os.path.join(base_dir, split, label)
            num_frames = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))])
            print(f"  {label.capitalize()} Frames: {num_frames}")

# Count frames in flattened folders
count_flattened_frames(frames_base_dir)



Counting frames in 'train_flattened' folder:
  Real Frames: 29163
  Fake Frames: 57372

Counting frames in 'test_flattened' folder:
  Real Frames: 7116
  Fake Frames: 12939


# Here we reduced the samples

In [24]:
import os
import random
import shutil

def reduce_samples(input_folder, output_folder, num_samples):
    """
    Randomly select 'num_samples' frames from the input_folder and copy them to the output_folder.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all frame files
    all_frames = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))]
    print(f"Total frames in '{input_folder}': {len(all_frames)}")

    # Randomly sample 'num_samples' frames
    selected_frames = random.sample(all_frames, min(num_samples, len(all_frames)))

    # Copy selected frames to the output folder
    for frame in selected_frames:
        src = os.path.join(input_folder, frame)
        dst = os.path.join(output_folder, frame)
        shutil.copy(src, dst)

    print(f"Copied {len(selected_frames)} frames to '{output_folder}'.")

# Paths to flattened directories
train_real_input = '/content/data/frames/train_flattened/real'
train_fake_input = '/content/data/frames/train_flattened/fake'
test_real_input = '/content/data/frames/test_flattened/real'
test_fake_input = '/content/data/frames/test_flattened/fake'

# Output directories for reduced dataset
train_real_output = '/content/data/frames_reduced/train/real'
train_fake_output = '/content/data/frames_reduced/train/fake'
test_real_output = '/content/data/frames_reduced/test/real'
test_fake_output = '/content/data/frames_reduced/test/fake'

# Reduce the dataset
reduce_samples(train_real_input, train_real_output, num_samples=5000)
reduce_samples(train_fake_input, train_fake_output, num_samples=5000)
reduce_samples(test_real_input, test_real_output, num_samples=1000)
reduce_samples(test_fake_input, test_fake_output, num_samples=1000)


Total frames in '/content/data/frames/train_flattened/real': 29163
Copied 5000 frames to '/content/data/frames_reduced/train/real'.
Total frames in '/content/data/frames/train_flattened/fake': 57372
Copied 5000 frames to '/content/data/frames_reduced/train/fake'.
Total frames in '/content/data/frames/test_flattened/real': 7116
Copied 1000 frames to '/content/data/frames_reduced/test/real'.
Total frames in '/content/data/frames/test_flattened/fake': 12939
Copied 1000 frames to '/content/data/frames_reduced/test/fake'.


# Verification for our number of samples

In [25]:
def count_reduced_samples(base_folder):
    for split in ['train', 'test']:
        print(f"\nCounting samples in '{split}' folder:")
        for label in ['real', 'fake']:
            folder_path = os.path.join(base_folder, split, label)
            num_files = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))])
            print(f"  {label.capitalize()} Frames: {num_files}")

# Count frames in reduced dataset
count_reduced_samples('/content/data/frames_reduced')



Counting samples in 'train' folder:
  Real Frames: 5000
  Fake Frames: 5000

Counting samples in 'test' folder:
  Real Frames: 1000
  Fake Frames: 1000


# Zipping and downloading these reduced datasets, as now the dataset has been processed and perpared now ti will be fed to our BNN pre trained model for training.

In [26]:
import shutil
import os

# Define paths to reduced dataset folders
reduced_train_folder = '/content/data/frames_reduced/train'
reduced_test_folder = '/content/data/frames_reduced/test'

# Define output zip file paths
train_zip_path = '/content/reduced_train.zip'
test_zip_path = '/content/reduced_test.zip'

# Zip the reduced train folder
shutil.make_archive(train_zip_path.replace('.zip', ''), 'zip', reduced_train_folder)
print(f"Train dataset zipped at: {train_zip_path}")

# Zip the reduced test folder
shutil.make_archive(test_zip_path.replace('.zip', ''), 'zip', reduced_test_folder)
print(f"Test dataset zipped at: {test_zip_path}")


Train dataset zipped at: /content/reduced_train.zip
Test dataset zipped at: /content/reduced_test.zip
